# 03 — Time Series, Periods & Transits

## What you’ll learn
- Why cadence and gaps matter
- Detrending a light curve
- Period search with **Lomb–Scargle** (works for uneven sampling)
- Phase folding
- A simple **box transit** search (toy BLS)

We’ll simulate data, then recover the injected period.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import lombscargle

np.random.seed(2)


## 1) Simulate uneven observation times

Real astronomy time series often have gaps:
- daytime / weather
- scheduling constraints


In [ ]:
n = 400
# irregular sampling over 30 days
t = np.sort(np.random.uniform(0, 30.0, size=n))
# remove a big gap to mimic weather
mask = ~((t>12) & (t<15))
t = t[mask]
print("N points:", len(t), "time span:", (t.min(), t.max()))


## 2) Simulate a periodic variable + noise

We'll make a sinusoid (e.g., pulsating star) and optionally add a transit-like dip.


In [ ]:
P_true = 2.345  # days
amp = 0.02      # relative amplitude (2%)
sigma = 0.01    # noise

y = 1.0 + amp*np.sin(2*np.pi*t/P_true)
y += np.random.normal(0, sigma, size=len(t))

plt.figure(figsize=(7,3.5))
plt.plot(t, y, ".", markersize=3)
plt.xlabel("time [days]")
plt.ylabel("relative flux")
plt.title("Simulated light curve")
plt.tight_layout()
plt.show()


## 3) Detrending (very simple)

Detrending means removing slow trends (instrument / atmosphere).  
Here we'll subtract a running median.


In [ ]:
def running_median(x, window=31):
    # simple running median using padding (OK for teaching)
    w = int(window)
    if w % 2 == 0: w += 1
    k = w//2
    out = np.empty_like(x)
    for i in range(len(x)):
        lo = max(0, i-k)
        hi = min(len(x), i+k+1)
        out[i] = np.median(x[lo:hi])
    return out

trend = running_median(y, window=41)
y_d = y / trend  # divide to remove multiplicative trend

plt.figure(figsize=(7,3.5))
plt.plot(t, y, ".", markersize=3, label="raw")
plt.plot(t, trend, "-", linewidth=2, label="trend")
plt.xlabel("time [days]")
plt.ylabel("flux")
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(7,3.5))
plt.plot(t, y_d, ".", markersize=3)
plt.xlabel("time [days]")
plt.ylabel("detrended flux")
plt.title("Detrended light curve")
plt.tight_layout()
plt.show()


## 4) Lomb–Scargle periodogram

SciPy's `lombscargle` expects angular frequency (rad/time).  
We'll scan frequencies, then pick the best period.


In [ ]:
# frequency grid (cycles/day)
fmin, fmax = 0.1, 5.0
freq = np.linspace(fmin, fmax, 5000)
ang = 2*np.pi*freq

# lombscargle requires mean-subtracted signal
yy = y_d - np.mean(y_d)

pgram = lombscargle(t, yy, ang, precenter=False, normalize=True)

best_idx = np.argmax(pgram)
f_best = freq[best_idx]
P_best = 1.0 / f_best

print("True period:", P_true, "Recovered:", P_best)

plt.figure(figsize=(7,3.5))
plt.plot(1/freq, pgram, linewidth=1)
plt.axvline(P_true, linestyle="--", label="true")
plt.axvline(P_best, linestyle="-", label="best")
plt.gca().invert_xaxis()
plt.xlabel("period [days]")
plt.ylabel("power")
plt.title("Lomb–Scargle periodogram")
plt.legend()
plt.tight_layout()
plt.show()


## 5) Phase folding

Phase folding overlays cycles:
\[
\phi = \mathrm{frac}(t/P)
\]


In [ ]:
def phase_fold(t, y, P, t0=0.0):
    phase = ((t - t0) / P) % 1.0
    idx = np.argsort(phase)
    return phase[idx], y[idx]

phase, yf = phase_fold(t, y_d, P_best)

plt.figure(figsize=(7,3.5))
plt.plot(phase, yf, ".", markersize=3, alpha=0.8)
plt.xlabel("phase")
plt.ylabel("detrended flux")
plt.title("Phase-folded light curve")
plt.tight_layout()
plt.show()


## 6) (Optional) Add a transit and do a toy box search

A transit looks like a periodic dip.  
Real searches use BLS; here we'll do a simple brute-force scan.


In [ ]:
# Inject a transit into the *existing* detrended data for demonstration
P_tr = 3.7
depth = 0.03     # 3%
dur = 0.12       # days (box half-width ~ dur/2)

def inject_transit(t, y, P, depth=0.02, dur=0.1, t0=0.2):
    y2 = y.copy()
    phase = ((t - t0) / P) % 1.0
    in_tr = (phase < dur/P) | (phase > 1 - dur/P)
    y2[in_tr] *= (1.0 - depth)
    return y2

y_tr = inject_transit(t, y_d, P_tr, depth=depth, dur=dur)

plt.figure(figsize=(7,3.5))
plt.plot(t, y_tr, ".", markersize=3)
plt.xlabel("time [days]")
plt.ylabel("flux")
plt.title("Light curve with injected transit")
plt.tight_layout()
plt.show()


In [ ]:
def box_score(t, y, P, dur, t0=0.0):
    # compute mean in-transit and out-of-transit; score = depth / noise
    phase = ((t - t0) / P) % 1.0
    in_tr = (phase < dur/P) | (phase > 1 - dur/P)
    if in_tr.sum() < 5 or (~in_tr).sum() < 5:
        return -np.inf, np.nan
    yin = np.mean(y[in_tr])
    yout = np.mean(y[~in_tr])
    depth_est = yout - yin
    noise = np.std(y[~in_tr])
    score = depth_est / (noise + 1e-12)
    return score, depth_est

P_grid = np.linspace(1.0, 8.0, 2000)
scores = []
depths = []
for P in P_grid:
    s,d = box_score(t, y_tr, P, dur=dur, t0=0.2)
    scores.append(s); depths.append(d)
scores = np.array(scores); depths = np.array(depths)

best = np.argmax(scores)
P_box = P_grid[best]
print("Injected transit period:", P_tr, "Recovered:", P_box)

plt.figure(figsize=(7,3.5))
plt.plot(P_grid, scores, linewidth=1)
plt.axvline(P_tr, linestyle="--", label="injected")
plt.axvline(P_box, linestyle="-", label="best")
plt.xlabel("period [days]")
plt.ylabel("box score (toy)")
plt.title("Toy transit period scan")
plt.legend()
plt.tight_layout()
plt.show()


## Try it
- Increase noise and see when the period becomes ambiguous.
- Make the sampling *more* gappy and see how alias peaks appear.
- Inject multiple signals and see what the periodogram finds.

Next notebook: **04 — Spectra: Lines & Redshift**
